Análise inicial

In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [ ]:
df = pd.read_csv('../data/raw/dados.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
df.value_counts()

In [ ]:
df.describe(include='object')

___

Preprocessamento e Limpeza

In [ ]:
df.rename(columns={
    'sql_registration_number': 'id_iptu', # Número que individualiza o imóvel no cadastro do IPTU (Cadastro Imobiliário Fiscal)
    'transaction_nature': 'natureza_transacao', # Indica o tipo de negócio jurídico por meio do qual o imóvel está sendo transmitido
    'transaction_value_declared': 'valor_transacao', # Valor econômico que está sendo dado pela aquisição do imóvel
    'transaction_date': 'data_transacao', # Data do instrumento particular ou escritura pública por meio da qual o negócio jurídico está sendo formalizado
    'reference_market_value': 'valor_ref_mercado', # Valor divulgado pela Prefeitura, que servirá como parâmetro inicial no processo de obtenção da base de cálculo do ITBI
    'transmitted_proportion': 'proporcao_transmitida', # Parte do imóvel (em percentual) que está sendo transmitido
    'proportional_reference_market_value': 'valor_ref_proporcional', # Valor que será comparado com o Valor de Transação para se chegar à base de cálculo do ITBI
    'adopted_calculation_base': 'base_calculo_ITBI', # Base de Cálculo do ITBI
    'financing_type': 'tipo_financiamento', # Modalidade de financiamento vinculado à aquisição do imóvel
    'financed_value': 'valor_financiado', # Valor do financiamento concedido pelo banco ou instituição financeira para aquisição do imóvel
    'registry_office': 'cartorio', # Cartório de Registro de Imóvel responsável pela matrícula (ou transcrição) do imóvel transacionado
    'property_registration': 'matricula_imovel', # Número da matrícula (ou transcrição) do imóvel transacionado
    'sql_status': 'status_IPTU', # Sinaliza se determinado SQL está ativo ou cancelado no cadastro do IPTU (Cadastro Imobiliário Fiscal)
    'land_area_m2': 'area_terreno_m²', # Informa a área do terreno em m²
    'frontage_m': 'frente_m', # Medida, em metros, da frente do imóvel para o logradouro (rua, travessa, praça, avenida, etc.)
    'ideal_fraction': 'fracao_ideal', # Percentual atribuído a cada unidade autônoma em relação à área (de terreno) do condomínio
    'built_area_m2': 'area_construida_m²', # Informa a área construída em m²
    'iptu_use': 'cod_uso_IPTU', # Código relativo à finalidade preponderante a que o imóvel se destina
    'iptu_use_description': 'desc_uso_IPTU', # Identifica a finalidade preponderante a que o imóvel se destina
    'iptu_standard': 'cod_padrao_IPTU', # Código relativo ao tipo e padrão da construção
    'iptu_standard_description': 'desc_padrao_IPTU', # Identifica tipo relacionado ao código de padrão da construção
    'iptu_acc': 'ano_construcao', # Indica o ano do término da construção, corrigido
    'cep': 'cep', # CEP do logradouro do endereço do imóvel
    'number': 'numero_endereco', # Numeração do imóvel no seu respectivo logradouro (rua, travessa, praça, avenida, etc.)
    'address_type': 'tipo_logradouro', # Tipo do logradouro
    'address_name': 'nome_logradouro', # Nome do logradouro
    'address': 'endereco_completo', # Endereço completo com tipo e nome
    'state': 'uf', # Unidade federativa do Brasil
    'district': 'distrito', # Subdivisão do Município
    'zone': 'zona', # zona correspondente
    'lat': 'latitude', # Latitude do CEP
    'lng': 'longitude', # Longitude do CEP
    'city': 'cidade', # Cidade
    'city_ibge': 'cod_IBGE_cidade', # Códigos de Municípios do IBGE
    'ddd': 'ddd' # Código de Discagem Direta à Distância
}, inplace=True)

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(columns=['uf', 'cidade', 'cod_IBGE_cidade', 'ddd'], inplace=True)

In [ ]:
# transformando valor_transacao para uma natureza econômica

# 1. Cria a proporção em decimal (ex: 25 -> 0.25)
df['proporcao_transmitida'] = df['proporcao_transmitida'] / 100

# 2. Calcula o valor total (Novo Fato)
df['valor_imovel'] = df['valor_transacao'] / df['proporcao_transmitida']

df['valor_imovel'] = df['valor_imovel'].round(0)

In [ ]:
# Tratando os outliers de valor_transacao com o método Intervalo Interquartil(IQR)

# Calculo dos Quartis e o IQR
Q1 = df['valor_transacao'].quantile(0.25)
Q3 = df['valor_transacao'].quantile(0.75)
IQR = Q3 - Q1

# Calculo dos Limites
LIMITE_INFERIOR = Q1 - (1.5 * IQR)
LIMITE_SUPERIOR = Q3 + (1.5 * IQR)

# # Aplicar Capagem nos Outliers Superiores
# df.loc[df['valor_transacao'] > LIMITE_SUPERIOR, 'valor_transacao'] = LIMITE_SUPERIOR

# # Aplicar Capagem nos Outliers Inferiores
# df.loc[df['valor_transacao'] < LIMITE_INFERIOR, 'valor_transacao'] = LIMITE_INFERIOR

print(f"Q1 (25%): R$ {Q1:,.2f}")
print(f"Q3 (75%): R$ {Q3:,.2f}")
print(f"IQR: R$ {IQR:,.2f}")
print(f"Limite Superior (Outlier): R$ {LIMITE_SUPERIOR:,.2f}")

# Outliers
outliers_superiores = df[df['valor_transacao'] > LIMITE_SUPERIOR]
outliers_inferiores = df[df['valor_transacao'] < LIMITE_INFERIOR]

total_outliers = len(outliers_superiores) + len(outliers_inferiores)
print(f"\nTotal de outliers identificados (acima/abaixo): {total_outliers}")

tamanho_total = len(df)
percentual_outliers = (total_outliers / tamanho_total) * 100

print(f"Total de linhas a serem removidas: {total_outliers}")
print(f"Percentual de Outliers: {percentual_outliers:.2f}%")


In [ ]:
df = df[
    (df['valor_transacao'] >= LIMITE_INFERIOR) & 
    (df['valor_transacao'] <= LIMITE_SUPERIOR)
]

In [ ]:
colunas = [
    'tipo_financiamento',
    'desc_uso_IPTU',
    'tipo_logradouro',
    'nome_logradouro',
    'endereco_completo',
    'distrito',
    'zona'
]

valor = 'Não_informado'

df[colunas] = df[colunas].fillna(valor)

In [ ]:
mediana_ano_construcao = df['ano_construcao'].median()

# 2. Preencher os valores ausentes (NaN) com a Mediana
# O .astype(int) é usado para converter o resultado de volta para um número inteiro,
# já que anos são geralmente tratados como inteiros.
df['ano_construcao'] = df['ano_construcao'].fillna(mediana_ano_construcao)

In [ ]:
colunas = [
    'latitude',
    'longitude'
]

df[colunas] = df[colunas].fillna(0)

In [ ]:
placeholders = [
    '', ' ',
    'NÃO INFORMADO', 'NÃO SE APLICA', 'SEM INFORMACAO', 'Não_informado', 
    '-', '?', '#', '*',
    'N/A', 'NA', 'NULL', 'NI', 'nan', 'SI', 'S/I'
]

nulos = df.isnull()

inconsistencias = df.isin(placeholders)

todas_inconsistencias = (nulos | inconsistencias).sum()

total_inconsistencias = todas_inconsistencias.sum()

colunas = [
    'tipo_financiamento',
    'desc_uso_IPTU',
    'tipo_logradouro',
    'nome_logradouro',
    'endereco_completo',
    'distrito',
    'zona'
]

for coluna in colunas:
    mask_coluna = (nulos | inconsistencias)[coluna]
    valores = df.loc[mask_coluna, coluna]

    contagem = valores.value_counts(dropna=False)

    if contagem.empty:
        print(f'Nenhuma inconsistencia encontrada na coluna {coluna}')
    else:
        print(f'A coluna {coluna} possui {contagem.sum()} inconsitências')
        print(f'{contagem}\n')

In [ ]:
df['desc_uso_IPTU'] = df['desc_uso_IPTU'].replace(' ','Não_informado')

In [ ]:
status_a_remover = [
    'Cancelado Predial',
    'Cancelado Territorial',
    'Nâo_Existe_Exercicio'
]

df = df[~df['status_IPTU'].isin(status_a_remover)]

In [ ]:
df.shape

In [ ]:
df.to_parquet(
    '../data/processed/dados.parquet',
    index=False
)

___

Análise

# Dimensão Localização

- H1: Imóveis em bairros com maior Valor de Mercado de Referência (reference_market_value) declarado pela prefeitura apresentarão um transaction_value_declared proporcionalmente maior.

- H2: A distância a centros comerciais e eixos de transporte (que precisaria ser calculada usando as coordenadas do CEP) terá uma relação inversa com o preço: quanto mais perto do centro, mais caro.

- H3: Bairros com maior proporção de imóveis de alto padrão (identificáveis pela tipologia ou preço médio) terão uma distribuição de preços menos dispersa e com média mais alta.

# Dimensão Imóvel

- H4: A área construída (built_area_m2) será a feature com a maior correlação positiva com o transaction_value_declared.

- H5: Imóveis classificados como "Prédio de Apartamento" ou "Garagem (unidade autônoma)" (tipologias em address_type) terão um preço médio e um preço por metro quadrado significativamente diferentes de "Casa/Sobrado em condomínio".

- H6: A proporção transmitida (transmitted_proportion) afeta a complexidade da modelagem, mas não o preço total do imóvel: a previsão do modelo deve se concentrar no valor total, não apenas na fração vendida.

# Dimensão Tempo

- H7: O preço médio dos imóveis apresentou uma tendência de crescimento ao longo dos meses ou trimestres (Série Temporal) do período coberto pelos dados.

- H8: A variação do preço ao longo do tempo será mais volátil para imóveis de alto valor do que para imóveis de valor mais baixo (tendência de mercado).

In [ ]:
# Hipótese 1. Qual é a principal feature preditiva do preço de um imóvel em São Paulo, e qual o valor médio do metro quadrado por zona?


In [ ]:
# Hipótese 2. O valor de transacao real tende a ser superior ou inferior ao valor de referência do mercado declarado pela prefeitura? Em quais distritos essa diferença é maior?


In [ ]:
# Hipótese 3. Qual tipo de imóvel representa o maior valor de transação médio, e quais são os tipos mais frequentes nas transações?


In [ ]:
# Hipótese 4. Qual a taxa de valorização média anual dos imóveis no período analisado, considerando data da transação?


In [ ]:
# Hipótese 5. A maioria das transações representa a venda total do imóvel? Se não, como as transações parciais afetam a interpretação do valor da transação?

___

Modelagem

___

Análise Prescritiva

___